## Explore miyagi augmentations

This notebook is intended to explore what the pre-defined augmentations will do to your data.

In [ ]:
DATASET_DIR = "/home/gfuhr/projects/data/liveness/controlid_liveness_ds_5/IR/train"
RESIZE_SIZE = 192
BATCH_SIZE = 32
AUGMENTATION = "liveness_single"  # "noaug", "simple", "random_erase", "liveness_single"

import os, random
import torch
from PIL import Image
import numpy as np
import ipyplot
import augmentations

train_tf, _ = augmentations.get_augmentations(RESIZE_SIZE, AUGMENTATION)
noaug_tf, _ = augmentations.get_augmentations(RESIZE_SIZE, "noaug")

files = []
for root, _, fnames in os.walk(DATASET_DIR):
    for f in fnames:
        if f.lower().endswith((".jpg",".png",".jpeg")):
            files.append(os.path.join(root, f))

sample_files = random.sample(files, BATCH_SIZE)

imgs_aug = [train_tf(Image.open(f).convert("RGB")) for f in sample_files]
imgs_noaug = [noaug_tf(Image.open(f).convert("RGB")) for f in sample_files]

imgs_aug = torch.stack(imgs_aug)
imgs_noaug = torch.stack(imgs_noaug)

def denorm(img):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
    img = img*std + mean
    return (img.permute(1,2,0).numpy()*255).astype(np.uint8)

imgs_aug_np = [denorm(im) for im in imgs_aug]
imgs_noaug_np = [denorm(im) for im in imgs_noaug]

print("No augmentation:")
ipyplot.plot_images(imgs_noaug_np, [f"{i}_noaug" for i in range(BATCH_SIZE)], max_images=BATCH_SIZE)

print("With augmentation:")
ipyplot.plot_images(imgs_aug_np, [f"{i}_aug" for i in range(BATCH_SIZE)], max_images=BATCH_SIZE)
